In [ ]:
import numpy as np
import math

SR        = 16000
NFFT      = 512
MicNum    = 8
room_size = np.array([5.0, 5.0, 3.3])
c         = 343
r         = 0.045
Sor_dis   = [2.27055, 2.40018]
Sor_theta = [90, 270]
Sor_phi   = [18.75414, 21.38029]
DOA       = np.array( [Sor_theta, Sor_phi]).T # variable


def Sph2Cas2(angle):
    assert angle.ndim==2
    angle_tmp = angle.copy()*math.pi/180
    x = (np.cos(angle_tmp[:,0])*np.sin(angle_tmp[:,1]))[:,None]
    y = (np.sin(angle_tmp[:,0])*np.sin(angle_tmp[:,1]))[:,None]
    z = (np.cos(angle_tmp[:,1]))[:,None]
    return np.concatenate( (x, y, z), axis=1)


kappa           = Sph2Cas2(DOA) # variable
ff              = SR/NFFT * np.arange(0,int(NFFT//2)+1)   # (ff) fix
k               = 2*math.pi*ff / c                        # (ff) fix
# dMicStrc        = np.array([[ 0., 0., 0.], [-0.0325, 0.0325, 0.], [-0.065, 0., 0.], [-0.0325, -0.0325, 0.]]) # fix
dMicStrc = np.array([
    [ 0.0000000000,  0.0000000000,  0.0000000000],  # 0°
    [-0.0131801948,  0.0318198052,  0.0000000000],  # 45°
    [-0.0450000000,  0.0450000000,  0.0000000000],  # 90°
    [-0.0768198052,  0.0318198052,  0.0000000000],  # 135°
    [-0.0900000000,  0.0000000000,  0.0000000000],  # 180°
    [-0.0768198052, -0.0318198052,  0.0000000000],  # 225°
    [-0.0450000000, -0.0450000000,  0.0000000000],  # 270°
    [-0.0131801948, -0.0318198052,  0.0000000000],  # 315°
])


Af              = np.exp(1j*k[None, None, :]*((kappa @ dMicStrc.T)[:,:,None]))  # (sor, mic, ff), free field RTF
print(Af)

[[[ 1.        +0.j          1.        +0.j
    1.        +0.j         ...  1.        +0.j
    1.        +0.j          1.        +0.j        ]
  [ 1.        +0.j          0.99999106+0.00422955j
    0.99996422+0.00845903j ...  0.47633935+0.87926152j
    0.4726162 +0.88126836j  0.4688846 +0.88325943j]
  [ 1.        +0.j          0.99998211+0.00598147j
    0.99992844+0.01196273j ...  0.05147028+0.99867453j
    0.04549582+0.99896453j  0.03951973+0.99921879j]
  ...
  [ 1.        +0.j          0.99999106-0.00422955j
    0.99996422-0.00845903j ...  0.47633935-0.87926152j
    0.4726162 -0.88126836j  0.4688846 -0.88325943j]
  [ 1.        +0.j          0.99998211-0.00598147j
    0.99992844-0.01196273j ...  0.05147028-0.99867453j
    0.04549582-0.99896453j  0.03951973-0.99921879j]
  [ 1.        +0.j          0.99999106-0.00422955j
    0.99996422-0.00845903j ...  0.47633935-0.87926152j
    0.4726162 -0.88126836j  0.4688846 -0.88325943j]]

 [[ 1.        +0.j          1.        +0.j
    1.        +0.

In [4]:
RTF = np.transpose(Af, (2, 1, 0))       # (freq, mic, sor)
f   = RTF.shape[0]
SorNum = RTF.shape[2]

pinvA = [np.linalg.inv(RTF[ii].conj().T @ RTF[ii] + 1e-2 * np.eye(SorNum)) @ RTF[ii].conj().T
        for ii in range(f)]

print(pinvA)
print(RTF.shape)          # (257, 8, 2)
print(pinvA[0].shape)     # (2, 8)


[array([[0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j,
        0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j],
       [0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j,
        0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j, 0.06246096+0.j]]), array([[0.06246223-6.81525048e-17j, 0.06258534-4.37013528e-01j,
        0.06270846-6.18028336e-01j, 0.06258534-4.37013528e-01j,
        0.06246223+8.79223476e-17j, 0.06258534+4.37013528e-01j,
        0.06270846+6.18028336e-01j, 0.06258534+4.37013528e-01j],
       [0.06246223+6.81525048e-17j, 0.06233784+4.37048901e-01j,
        0.06221346+6.18078360e-01j, 0.06233784+4.37048901e-01j,
        0.06246223-8.80322370e-17j, 0.06233784-4.37048901e-01j,
        0.06221346-6.18078360e-01j, 0.06233784-4.37048901e-01j]]), array([[0.06246605-1.22759901e-16j, 0.06291568-7.98433971e-01j,
        0.06336531-1.12914064e+00j, 0.06291568-7.98433971e-01j,
        0.06246605+1.62387039e-16j, 0.06291568+7.98433971e-01j,